In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import numpy as np

In [5]:
file_url = 'https://raw.githubusercontent.com/aso-uts/applied_ds/master/unit3/dataset/Car%20Evaluation.csv'

In [6]:
df = pd.read_csv(file_url)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   buying_price      1728 non-null   object
 1   maintenance_cost  1728 non-null   object
 2   doors             1728 non-null   object
 3   persons_capacity  1728 non-null   object
 4   luggage_boot      1728 non-null   object
 5   safety            1728 non-null   object
 6   evaluation        1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [8]:
df.to_csv('../data/raw/car_evaluation.csv', index=False)

In [9]:
df_cleaned = df.copy()

In [10]:
cats_dict = {
    'buying_price': [['low', 'med', 'high', 'vhigh']],
    'maintenance_cost': [['low', 'med', 'high', 'vhigh']],
    'doors': [['2', '3', '4', '5more']],
    'persons_capacity': [['2', '4', 'more']],
    'luggage_boot': [['small', 'med', 'big']],
    'safety': [['low', 'med', 'high']],
    'evaluation': [['unacc', 'acc', 'good', 'vgood']],
}

In [11]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

In [12]:
for col, cats in cats_dict.items():
    col_encoder = OrdinalEncoder(categories=cats)
    df_cleaned[col] = col_encoder.fit_transform(df_cleaned[[col]])

In [13]:
num_cols = ['buying_price', 'maintenance_cost', 'doors', 'persons_capacity', 'luggage_boot', 'safety']

In [14]:
sc = StandardScaler()

In [15]:
df_cleaned[num_cols] = sc.fit_transform(df_cleaned[num_cols])

In [16]:
df_cleaned['evaluation'] = df_cleaned['evaluation'].astype(int)

In [17]:
X = pd.DataFrame(df_cleaned.copy())
df_cleaned.tail()

,buying_price,maintenance_cost,doors,persons_capacity,luggage_boot,safety,evaluation
1723,-1.341641,-1.341641,1.341641,1.224745,0.000000,0.000000,2
1724,-1.341641,-1.341641,1.341641,1.224745,0.000000,1.224745,3
1725,-1.341641,-1.341641,1.341641,1.224745,1.224745,-1.224745,0
1726,-1.341641,-1.341641,1.341641,1.224745,1.224745,0.000000,2
1727,-1.341641,-1.341641,1.341641,1.224745,1.224745,1.224745,3


In [19]:
import os
os.chdir(r"C:\Users\Angus\Documents\UTS MDSI\Advanced DSI\Projects\advdsi_at2\src\data")
from sets import subset_x_y, split_sets_by_time, save_sets, split_sets_random
os.chdir(r"C:\Users\Angus\Documents\UTS MDSI\Advanced DSI\Projects\advdsi_at2\notebooks") 

In [21]:
# Solution
X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(X, target_col='evaluation', test_ratio=0.2, to_numpy=True)


In [22]:
X_train.shape

(1036, 6)

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [25]:
os.chdir(r"C:\Users\Angus\Documents\UTS MDSI\Advanced DSI\Projects\advdsi_at2\src\models")
from pytorch import PytorchDataset
os.chdir(r"C:\Users\Angus\Documents\UTS MDSI\Advanced DSI\Projects\advdsi_at2\notebooks")

In [26]:
train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

In [27]:
os.chdir(r"C:\Users\Angus\Documents\UTS MDSI\Advanced DSI\Projects\advdsi_at2\src\models")
from null import NullModel
from performance import print_class_perf
from MClasspytorch import PytorchMultiClass, train_classification, test_classification
os.chdir(r"C:\Users\Angus\Documents\UTS MDSI\Advanced DSI\Projects\advdsi_at2\notebooks")

In [28]:
baseline_model = NullModel(target_type='classification')
y_base = baseline_model.fit_predict(y_train)

In [29]:
print_class_perf(y_base, y_train, set_name='Training', average='weighted')

Accuracy Training: 0.6988416988416989
F1 Training: 0.5749561249561249


In [30]:
model = PytorchMultiClass(X_train.shape[1])

In [31]:
### awkward fix
device = torch.device('cpu')

In [32]:
model.to(device)

PytorchMultiClass(
  (layer_1): Linear(in_features=6, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)

In [33]:
print(model)

PytorchMultiClass(
  (layer_1): Linear(in_features=6, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)


In [34]:
criterion = nn.CrossEntropyLoss()

In [35]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [36]:
N_EPOCHS = 50
BATCH_SIZE = 32

In [37]:
for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

c:\users\angus\appdata\local\programs\python\python38\lib\site-packages\torch\autograd\__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


Epoch: 0
	(train)	|	Loss: 0.0316	|	Acc: 74.2%
	(valid)	|	Loss: 0.0302	|	Acc: 78.9%
Epoch: 1
	(train)	|	Loss: 0.0295	|	Acc: 81.5%
	(valid)	|	Loss: 0.0284	|	Acc: 84.7%
Epoch: 2
	(train)	|	Loss: 0.0286	|	Acc: 84.5%
	(valid)	|	Loss: 0.0283	|	Acc: 85.3%
Epoch: 3
	(train)	|	Loss: 0.0293	|	Acc: 82.2%
	(valid)	|	Loss: 0.0280	|	Acc: 86.7%
Epoch: 4
	(train)	|	Loss: 0.0282	|	Acc: 85.8%
	(valid)	|	Loss: 0.0288	|	Acc: 83.5%
Epoch: 5
	(train)	|	Loss: 0.0288	|	Acc: 83.5%
	(valid)	|	Loss: 0.0280	|	Acc: 86.4%
Epoch: 6
	(train)	|	Loss: 0.0281	|	Acc: 86.1%
	(valid)	|	Loss: 0.0276	|	Acc: 87.6%
Epoch: 7
	(train)	|	Loss: 0.0283	|	Acc: 85.6%
	(valid)	|	Loss: 0.0291	|	Acc: 82.7%
Epoch: 8
	(train)	|	Loss: 0.0285	|	Acc: 84.6%
	(valid)	|	Loss: 0.0274	|	Acc: 88.2%
Epoch: 9
	(train)	|	Loss: 0.0283	|	Acc: 85.4%
	(valid)	|	Loss: 0.0277	|	Acc: 87.3%
Epoch: 10
	(train)	|	Loss: 0.0283	|	Acc: 86.0%
	(valid)	|	Loss: 0.0274	|	Acc: 88.2%
Epoch: 11
	(train)	|	Loss: 0.0282	|	Acc: 86.1%
	(valid)	|	Loss: 0.0276	|	Acc: 87.6%
Ep

In [58]:
torch.save(model, "../models/pytorch_multi_car_evaluation.pt")

In [59]:
test_loss, test_acc = test_classification(test_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)
print(f'\tLoss: {test_loss:.4f}\t|\tAccuracy: {test_acc:.1f}')

	Loss: 0.0307	|	Accuracy: 0.8
